# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 7 2023 9:52AM,256812,21,640399,"NBTY Global, Inc.",Released
1,Feb 7 2023 9:51AM,256810,10,MSP218275,"Methapharm, Inc.",Released
2,Feb 7 2023 9:51AM,256810,10,MSP218269,"Methapharm, Inc.",Released
3,Feb 7 2023 9:51AM,256810,10,MSP218271,"Methapharm, Inc.",Released
4,Feb 7 2023 9:51AM,256810,10,MSP218273,"Methapharm, Inc.",Released
5,Feb 7 2023 9:51AM,256810,10,MSP218276,"Methapharm, Inc.",Released
6,Feb 7 2023 9:50AM,256809,10,0086362009,ISDIN Corporation,Released
7,Feb 7 2023 9:50AM,256809,10,0086362006,ISDIN Corporation,Released
8,Feb 7 2023 9:50AM,256809,10,0086362007,ISDIN Corporation,Released
9,Feb 7 2023 9:50AM,256809,10,0086362008,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,256807,Released,1
29,256808,Released,1
30,256809,Released,9
31,256810,Released,5
32,256812,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256807,NaN,NaN,1.0
256808,NaN,NaN,1.0
256809,NaN,NaN,9.0
256810,NaN,NaN,5.0
256812,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256686,0.0,1.0,0.0
256711,1.0,0.0,1.0
256715,0.0,0.0,55.0
256740,0.0,0.0,1.0
256750,0.0,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256686,0,1,0
256711,1,0,1
256715,0,0,55
256740,0,0,1
256750,0,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256686,0,1,0
1,256711,1,0,1
2,256715,0,0,55
3,256740,0,0,1
4,256750,0,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256686,,1,
1,256711,1,,1
2,256715,,,55
3,256740,,,1
4,256750,,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 7 2023 9:52AM,256812,21,"NBTY Global, Inc."
1,Feb 7 2023 9:51AM,256810,10,"Methapharm, Inc."
6,Feb 7 2023 9:50AM,256809,10,ISDIN Corporation
15,Feb 7 2023 9:48AM,256808,21,"NBTY Global, Inc."
16,Feb 7 2023 9:47AM,256807,21,"NBTY Global, Inc."
17,Feb 7 2023 9:42AM,256805,16,TASA USA Inc.
18,Feb 7 2023 9:34AM,256804,19,"Graystone, LLC"
19,Feb 7 2023 9:27AM,256803,10,"Digital Brands, LLC"
22,Feb 7 2023 9:14AM,256782,15,"Alliance Pharma, Inc."
127,Feb 7 2023 8:52AM,256798,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 7 2023 9:52AM,256812,21,"NBTY Global, Inc.",,,1
1,Feb 7 2023 9:51AM,256810,10,"Methapharm, Inc.",,,5
2,Feb 7 2023 9:50AM,256809,10,ISDIN Corporation,,,9
3,Feb 7 2023 9:48AM,256808,21,"NBTY Global, Inc.",,,1
4,Feb 7 2023 9:47AM,256807,21,"NBTY Global, Inc.",,,1
5,Feb 7 2023 9:42AM,256805,16,TASA USA Inc.,,,1
6,Feb 7 2023 9:34AM,256804,19,"Graystone, LLC",,,1
7,Feb 7 2023 9:27AM,256803,10,"Digital Brands, LLC",,,3
8,Feb 7 2023 9:14AM,256782,15,"Alliance Pharma, Inc.",,,105
9,Feb 7 2023 8:52AM,256798,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 9:52AM,256812,21,"NBTY Global, Inc.",1,,
1,Feb 7 2023 9:51AM,256810,10,"Methapharm, Inc.",5,,
2,Feb 7 2023 9:50AM,256809,10,ISDIN Corporation,9,,
3,Feb 7 2023 9:48AM,256808,21,"NBTY Global, Inc.",1,,
4,Feb 7 2023 9:47AM,256807,21,"NBTY Global, Inc.",1,,
5,Feb 7 2023 9:42AM,256805,16,TASA USA Inc.,1,,
6,Feb 7 2023 9:34AM,256804,19,"Graystone, LLC",1,,
7,Feb 7 2023 9:27AM,256803,10,"Digital Brands, LLC",3,,
8,Feb 7 2023 9:14AM,256782,15,"Alliance Pharma, Inc.",105,,
9,Feb 7 2023 8:52AM,256798,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 9:52AM,256812,21,"NBTY Global, Inc.",1,,
1,Feb 7 2023 9:51AM,256810,10,"Methapharm, Inc.",5,,
2,Feb 7 2023 9:50AM,256809,10,ISDIN Corporation,9,,
3,Feb 7 2023 9:48AM,256808,21,"NBTY Global, Inc.",1,,
4,Feb 7 2023 9:47AM,256807,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 9:52AM,256812,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Feb 7 2023 9:51AM,256810,10,"Methapharm, Inc.",5.0,NaN,NaN
2,Feb 7 2023 9:50AM,256809,10,ISDIN Corporation,9.0,NaN,NaN
3,Feb 7 2023 9:48AM,256808,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 7 2023 9:47AM,256807,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 9:52AM,256812,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Feb 7 2023 9:51AM,256810,10,"Methapharm, Inc.",5.0,0.0,0.0
2,Feb 7 2023 9:50AM,256809,10,ISDIN Corporation,9.0,0.0,0.0
3,Feb 7 2023 9:48AM,256808,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 7 2023 9:47AM,256807,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4621913,255.0,6.0,2.0
15,256782,105.0,0.0,0.0
16,513491,1.0,1.0,0.0
19,1540705,4.0,4.0,0.0
20,256788,1.0,0.0,0.0
21,770427,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4621913,255.0,6.0,2.0
1,15,256782,105.0,0.0,0.0
2,16,513491,1.0,1.0,0.0
3,19,1540705,4.0,4.0,0.0
4,20,256788,1.0,0.0,0.0
5,21,770427,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,255.0,6.0,2.0
1,15,105.0,0.0,0.0
2,16,1.0,1.0,0.0
3,19,4.0,4.0,0.0
4,20,1.0,0.0,0.0
5,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,255.0
1,15,Released,105.0
2,16,Released,1.0
3,19,Released,4.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,1.0,4.0,0.0,0.0
Released,255.0,105.0,1.0,4.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,1.0,4.0,0.0,0.0
2,Released,255.0,105.0,1.0,4.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,1.0,4.0,0.0,0.0
2,Released,255.0,105.0,1.0,4.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()